# Homework 1

## Question 1. Understanding docker first run

Run docker with the python:3.12.8 image in an interactive mode, use the entrypoint bash.

What's the version of pip in the image?

I created a Docker container from the following Dockerfile:
```Dockerfile
FROM python:3.12.8

RUN pip install pandas

ENTRYPOINT [ "bash" ]

Then I ran `pip --version` and got `24.3.1`.

## Question 2. Understanding Docker networking and docker-compose
Given the following docker-compose.yaml, what is the hostname and port that pgadmin should use to connect to the postgres database?

```yaml
services:
  db:
    container_name: postgres
    image: postgres:17-alpine
    environment:
      POSTGRES_USER: 'postgres'
      POSTGRES_PASSWORD: 'postgres'
      POSTGRES_DB: 'ny_taxi'
    ports:
      - '5433:5432'
    volumes:
      - vol-pgdata:/var/lib/postgresql/data

  pgadmin:
    container_name: pgadmin
    image: dpage/pgadmin4:latest
    environment:
      PGADMIN_DEFAULT_EMAIL: "pgadmin@pgadmin.com"
      PGADMIN_DEFAULT_PASSWORD: "pgadmin"
    ports:
      - "8080:80"
    volumes:
      - vol-pgadmin_data:/var/lib/pgadmin  

volumes:
  vol-pgdata:
    name: vol-pgdata
  vol-pgadmin_data:
    name: vol-pgadmin_data

The hostname is `db` and the port is `5432`.

## Prepare Postgres

Run Postgres and load data as shown in the videos We'll use the green taxi trips from October 2019:

`wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-10.csv.gz`

You will also need the dataset with zones:

`wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv`

Download this data and put it into Postgres.

You can use the code from the course. It's up to you whether you want to use Jupyter or a python script.

## Question 3. Trip Segmentation Count

During the period of October 1st 2019 (inclusive) and November 1st 2019 (exclusive), how many trips, respectively, happened:

    Up to 1 mile
    In between 1 (exclusive) and 3 miles (inclusive),
    In between 3 (exclusive) and 7 miles (inclusive),
    In between 7 (exclusive) and 10 miles (inclusive),
    Over 10 miles


In [1]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    port="5433",
    database="ny_taxi",
    user="postgres",
    password="postgres"
)

cur = conn.cursor()

In [2]:
q3_query = """SELECT
    COUNT(CASE WHEN trip_distance <= 1 THEN 1 END) AS trips_up_to_1_mile,
    COUNT(CASE WHEN trip_distance > 1 AND trip_distance <= 3 THEN 1 END) AS trips_between_1_and_3_miles,
    COUNT(CASE WHEN trip_distance > 3 AND trip_distance <= 7 THEN 1 END) AS trips_between_3_and_7_miles,
    COUNT(CASE WHEN trip_distance > 7 AND trip_distance <= 10 THEN 1 END) AS trips_between_7_and_10_miles,
    COUNT(CASE WHEN trip_distance > 10 THEN 1 END) AS trips_over_10_miles
FROM
    green_tripdata
WHERE
    lpep_dropoff_datetime >= '2019-10-01'
    AND lpep_dropoff_datetime < '2019-11-01';
"""

In [3]:
cur.execute(q3_query)
rows = cur.fetchall()
print(rows)

[(104802, 198924, 109603, 27678, 35189)]


## Question 4. Longest trip for each day

Which was the pick up day with the longest trip distance? Use the pick up time for your calculations.

Tip: For every day, we only care about one single trip with the longest distance.

In [4]:
q4_query = """SELECT
    DATE(lpep_pickup_datetime) AS pickup_day,
    MAX(trip_distance) AS longest_trip_distance
FROM
    green_tripdata
GROUP BY
    pickup_day
ORDER BY
    longest_trip_distance DESC
LIMIT 1;"""

In [5]:
cur.execute(q4_query)
rows = cur.fetchall()
print(rows)

[(datetime.date(2019, 10, 31), 515.89)]


## Question 5. Three biggest pickup zones

Which were the top pickup locations with over 13,000 in total_amount (across all trips) for 2019-10-18?

Consider only lpep_pickup_datetime when filtering by date.

In [6]:
q5_query = """SELECT
    PULocationID,
    SUM(total_amount) AS total_revenue
FROM
    green_tripdata
WHERE
    lpep_pickup_datetime::DATE = '2019-10-18'
GROUP BY
    PULocationID
HAVING
    SUM(total_amount) > 13000
ORDER BY
    total_revenue DESC;"""

In [7]:
cur.execute(q5_query)
rows = cur.fetchall()
print(rows)

[(74, 18686.680000000084), (75, 16797.26000000007), (166, 13029.790000000039)]


## Question 6. Largest tip

For the passengers picked up in October 2019 in the zone named "East Harlem North" which was the drop off zone that had the largest tip?

Note: it's tip , not trip|

We need the name of the zone, not the ID.

In [8]:
q6_query = """SELECT
    tz_dropoff.zone AS dropoff_zone,
    MAX(gtd.tip_amount) AS largest_tip
FROM
    green_tripdata gtd
JOIN
    taxi_zone_lookup tz_pickup ON gtd.PULocationID = tz_pickup.location_id
JOIN
    taxi_zone_lookup tz_dropoff ON gtd.DOLocationID = tz_dropoff.location_id
WHERE
    tz_pickup.zone = 'East Harlem North'
    AND gtd.lpep_pickup_datetime >= '2019-10-01'
    AND gtd.lpep_pickup_datetime < '2019-11-01'
GROUP BY
    tz_dropoff.zone
ORDER BY
    largest_tip DESC
LIMIT 1;
"""

In [9]:
cur.execute(q6_query)
rows = cur.fetchall()
print(rows)

[('JFK Airport', 87.3)]


In [10]:
# Close the connection
cur.close()
conn.close()

## Question 7. Terraform Workflow

Which of the following sequences, respectively, describes the workflow for:

    Downloading the provider plugins and setting up backend,
    Generating proposed changes and auto-executing the plan
    Remove all resources managed by terraform`


To do this, we need to perform

`terraform init` to download the provider plugins and set up backend,

`terraform apply -auto-approve` to generate proposed changes and auto-execute the plan, and

`terraform destroy` to remove all resources managed by terraform